<a href="https://colab.research.google.com/github/mel200401/yolo-python-project/blob/visualization/train_yolos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade kagglehub

Using Colab cache for faster access to the 'car-object-detection' dataset.
Path to dataset files: /kaggle/input/car-object-detection


In [16]:
import kagglehub
import shutil
import os

# Download (will use cache)
data_dir = kagglehub.dataset_download("sshikamaru/car-object-detection")

# Create target directory
target_dir = "/content"
os.makedirs(target_dir, exist_ok=True)

# Copy dataset
shutil.copytree(data_dir, target_dir, dirs_exist_ok=True)

print("Dataset copied to:", target_dir)

Using Colab cache for faster access to the 'car-object-detection' dataset.
Dataset copied to: /content


In [19]:
# Set Up Data Paths
from pathlib import Path

dataset_directory = Path(target_dir) / "data"
train_img_directory = dataset_directory / "training_images"
test_img_directory = dataset_directory / "testing_images"
labels_directory = dataset_directory / "labels"
labels_directory.mkdir(parents=True, exist_ok=True)

csv_path = dataset_directory / "train_solution_bounding_boxes (1).csv"

print("directories and path are set up")


directories and path are set up


In [20]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Image Sizes
img_width = 676
img_height = 380

class_id = 0  # single class: car, pictures without car do not have annotation (need to create empty files for label)

# create empty label files for images without cars
for folder in [train_img_directory, test_img_directory]:
    for img_path in folder.iterdir():
        if img_path.suffix.lower() == ".jpg":
            (labels_directory / f"{img_path.stem}.txt").touch()

print("empty labels are created")

# have to convert the file (PASCAL VOC annotation) to YOLO readable file
df = pd.read_csv(csv_path)

for _, row in df.iterrows():
    img_name = row["image"]
    xmin, ymin, xmax, ymax = row["xmin"], row["ymin"], row["xmax"], row["ymax"]

    # clipping in case of negative or values outside of the image
    xmin = max(0, xmin)
    ymin = max(0, ymin)
    xmax = min(img_width, xmax)
    ymax = min(img_height, ymax)

    # Convert to YOLO normalized format
    x_center = ((xmin + xmax) / 2) / img_width
    y_center = ((ymin + ymax) / 2) / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height

    label_path = labels_directory / f"{Path(img_name).stem}.txt"

    with label_path.open("a") as f:
        f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

print("YOLO label files created.")

empty labels are created
YOLO label files created.


In [21]:
# Splitting into training, evaluate and the test set
for split in ["train", "val"]:
    (dataset_directory / split / "images").mkdir(parents=True, exist_ok=True)
    (dataset_directory / split / "labels").mkdir(parents=True, exist_ok=True)

# List of training images (so we don't use the test images for the split)
train_images = [
    img for img in train_img_directory.iterdir() if img.suffix.lower() == ".jpg"
]

# 80% train, 20% val
train_imgs, val_imgs = train_test_split(train_images, test_size=0.2, random_state=42)

print("Data has been split")

# Copy training images + labels
for img_path in train_imgs:
    shutil.copy(img_path, dataset_directory / "train" / "images" / img_path.name)
    shutil.copy(
        labels_directory / f"{img_path.stem}.txt",
        dataset_directory / "train" / "labels" / f"{img_path.stem}.txt",
    )

# Copy validation images + labels
for img_path in val_imgs:
    shutil.copy(img_path, dataset_directory / "val" / "images" / img_path.name)
    shutil.copy(
        labels_directory / f"{img_path.stem}.txt",
        dataset_directory / "val" / "labels" / f"{img_path.stem}.txt",
    )

# have the test set also in same format in case we want to have some evaluation metrics
(dataset_directory / "test" / "images").mkdir(parents=True, exist_ok=True)
(dataset_directory / "test" / "labels").mkdir(parents=True, exist_ok=True)

# Copy test images + labels
for img_path in test_img_directory.iterdir():
    if img_path.suffix.lower() == ".jpg":
        shutil.copy(img_path, dataset_directory / "test" / "images" / img_path.name)
        shutil.copy(
            labels_directory / f"{img_path.stem}.txt",
            dataset_directory / "test" / "labels" / f"{img_path.stem}.txt",
        )

print("Dataset is ready for YOLO.")

# Create Yaml file for YOLO
yaml_content = f"""
path: {dataset_directory.resolve()}
train: train/images
val: val/images
test: test/images

nc: 1
names: ["car"]
"""

"""
train: {dataset_directory / "train" / "images"}
val: {dataset_directory / "val" / "images"}
test: {dataset_directory / "test" / "images"}
"""


with open(dataset_directory / "data.yaml", "w") as f:
    f.write(yaml_content)

print("data.yaml created.")


Data has been split
Dataset is ready for YOLO.
data.yaml created.


In [ ]:
!pip install ultralytics

In [24]:
from ultralytics import YOLO
import torch

#import model
model = YOLO('yolov8n.pt')

if torch.backends.mps.is_available():#for ios
    device_type = 'mps'
elif torch.cuda.is_available(): #for cuda
    device_type = '0'
else:
    device_type = 'cpu'

#start training
results = model.train(
    data='data/data.yaml',
    epochs=40,
    imgsz=640,
    batch=16,                #adjust this if you have small VRAM
    name='car_detection',
    device=device_type
)


Ultralytics 8.4.18 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=car_detection2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plo

In [26]:
from ultralytics import YOLO
import torch

#import model
model = YOLO('yolo26n.pt')

if torch.backends.mps.is_available():#for ios
    device_type = 'mps'
elif torch.cuda.is_available(): #for cuda
    device_type = '0'
else:
    device_type = 'cpu'

#start training
results = model.train(
    data='data/data.yaml',
    epochs=40,
    imgsz=640,
    batch=16,                #adjust this if you have small VRAM
    name='car_detection',
    device=device_type
)


Ultralytics 8.4.18 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=car_detection3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plo